# Uvod #

U ovoj lekciji ćemo vidjeti kako možemo izgraditi neuronske mreže sposobne naučiti složene vrste odnosa po kojima su duboke neuronske mreže poznate.

Ključna ideja ovdje je *modularnost*, izgradnja složene mreže od jednostavnijih funkcionalnih jedinica. Vidjeli smo kako linearna jedinica izračunava linearnu funkciju -- sada ćemo vidjeti kako kombinirati i modificirati te pojedinačne jedinice za modeliranje složenijih odnosa.

# Slojevi #

Neuronske mreže obično organiziraju svoje neurone u **slojeve**. Kada skupimo linearne jedinice koje imaju zajednički skup ulaza, dobivamo **gust** sloj.

<figure style="padding: 1em;">
<img src="figs/2MA4iMV.png" width="300" alt="A stack of three circles in an input layer connected to two circles in a dense layer.">
<figcaption style="textalign: center; font-style: italic"><center>Gusti sloj dviju linearnih jedinica koje primaju dva ulaza i bias.
</center></figcaption>
</figure>

Mogli biste zamisliti svaki sloj u neuronskoj mreži kao osobu koja izvodi neku vrstu relativno jednostavne transformacije. Kroz duboku hrpu slojeva, neuronska mreža može transformirati svoje ulaze na sve složenije načine. U dobro obučenoj neuronskoj mreži, svaki sloj je transformacija koja nas malo približava rješenju.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Mnoge vrste slojeva</strong><br>
"Sloj" u Kerasu je vrlo općenita stvar. Sloj može biti, u biti, bilo koja vrsta <em>transformacije podataka</em>. Mnogi slojevi, kao što su <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D">konvolucijski</a> i <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN">ponavljajućih</a> slojeva, transformiraju podatke upotrebom neurona i prvenstveno se razlikuju po obrascu veza koje tvore. Drugi se pak koriste za <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding">inženjering značajki</a> ili samo <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers/Add">jednostavna aritmetika</a>. Postoji cijeli svijet slojeva za otkrivanje -- <a href="https://www.tensorflow.org/api_docs/python/tf/keras/layers">bacit oko</a>!
</blockquote>

# Funkcija aktivacije #

Međutim, pokazalo se da dva gusta sloja bez ičega između nisu ništa bolja od jednog gustog sloja samog po sebi. Gusti nas slojevi sami po sebi nikada ne mogu pomaknuti iz svijeta linija i ravnina. Ono što nam treba je nešto *nelinearno*. Ono što nam treba su aktivacijske funkcije.

<figure style="padding: 1em;">
<img src="figs/OLSUEYT.png" width="400" alt=" ">
<figcaption style="textalign: center; font-style: italic"><center>Bez aktivacijskih funkcija, neuronske mreže mogu naučiti samo linearne odnose. Kako bismo prilagodili krivulje, morat ćemo koristiti aktivacijske funkcije.
</center></figcaption>
</figure>

**Aktivacijska funkcija** jednostavno je neka funkcija koju primjenjujemo na svaki od izlaza sloja (njegove *aktivacije*). Najčešća je funkcija *rectifier* $max(0, x)$.

<figure style="padding: 1em;">
<img src="figs/aeIyAlF.png" width="400" alt="A graph of the rectifier function. The line y=x when x>0 and y=0 when x<0, making a 'hinge' shape like '_/'.">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

Funkcija rectifier ima grafikon koji je linija s negativnim dijelom "ispravljenim" na nulu. Primjena funkcije na izlaze neurona stvorit će *zakrivljenje* u podacima, udaljavajući nas od jednostavnih linija.

Kada spojimo ispravljač na linearnu jedinicu, dobivamo **ispravljenu linearnu jedinicu** ili **ReLU**. (Iz tog razloga, uobičajeno je nazvati funkciju ispravljača "ReLU funkcija".) Primjena ReLU aktivacije na linearnu jedinicu znači da izlaz postaje `max(0, w * x + b)`, što bismo mogli nacrtati u dijagram poput:

<figure style="padding: 1em;">
<img src="figs/eFry7Yu.png" width="250" alt="Diagram of a single ReLU. Like a linear unit, but instead of a '+' symbol we now have a hinge '_/'. ">
<figcaption style="textalign: center; font-style: italic"><center>Rektificirana linearna jedinica - ReLU.
</center></figcaption>
</figure>

# Slaganje gustih slojeva #

Sada kada imamo nešto nelinearnosti, pogledajmo kako možemo složiti slojeve da bismo dobili složene transformacije podataka.

<figure style="padding: 1em;">
<img src="https://storage.googleapis.com/kaggle-media/learn/images/Y5iwFQZ.png" width="450" alt="An input layer, two hidden layers, and a final linear layer.">
<figcaption style="textalign: center; font-style: italic"><center>Snop gustih slojeva čini "potpuno povezanu" mrežu.
</center></figcaption>
</figure>

Slojevi prije izlaznog sloja ponekad se nazivaju **skriveni** jer nikad ne vidimo izravno njihove izlaze.

Sada primijetite da je završni (izlazni) sloj linearna jedinica (što znači da nema aktivacijske funkcije). To ovu mrežu čini prikladnom za zadatak regresije, gdje pokušavamo predvidjeti neku proizvoljnu numeričku vrijednost. Drugi zadaci (poput klasifikacije) mogu zahtijevati aktivacijsku funkciju na izlazu.

## Izrada sekvencijalnih modela ##

`Sekvencijalni` model koji smo koristili povezat će zajedno popis slojeva redom od prvog do zadnjeg: prvi sloj dobiva ulaz, zadnji sloj proizvodi izlaz. Ovo stvara model na gornjoj slici:

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    # the hidden ReLU layers
    layers.Dense(units=4, activation='relu', input_shape=[2]),
    layers.Dense(units=3, activation='relu'),
    # the linear output layer
    layers.Dense(units=1),
])

c:\Users\Perica\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Obavezno proslijedite sve slojeve zajedno na popisu, poput `[sloj, sloj, sloj, ...]`, umjesto kao zasebne argumente. Da biste sloju dodali funkciju aktivacije, samo dajte njezin naziv u argumentu `activation`.

# Tvoj zadatak #

Sada, [**kreiraj duboku neuronsku mrežu**](Deep_Neural_Networks_exercise_hr.ipynb) za skup podataka *Concrete*.